In [ ]:
import numpy as np
import torch
import time

In [ ]:
feats=np.random.normal(size=(5000,300))
protos=np.random.normal(size=(500,300))

def forloopdists(feats,protos):
  #shapes: are (N,D), (P,D)

  res = np.zeros((feats.shape[0], protos.shape[0]))

  for i in range(feats.shape[0]):
    for j in range(protos.shape[0]):
      res[i, j] = np.sum(np.power((feats[i, :] - protos[j, :]), 2))
  
  return res

dists = forloopdists(feats,protos)
print(dists.shape)

(5000, 500)


In [64]:
feats=np.random.normal(size=(5000,300))
protos=np.random.normal(size=(500,300))

def pytorchdists(feats0,protos0,device="cpu"):
  #shapes: are (N,D), (P,D)
  feats = torch.Tensor(feats0)
  protos = torch.Tensor(protos0)

  t1 = torch.sum(feats * feats, axis=1).unsqueeze(1) # shape [5000, 1]
  t2 = 2 * feats @ protos.t() # shape [5000, 500]
  t3 = torch.sum(protos * protos, axis=1).unsqueeze(0) # shape [1, 500]

  return t1 - t2 + t3

dists = pytorchdists(feats,protos)
print(dists.shape)

torch.Size([5000, 500])


In [60]:
feats=np.random.normal(size=(5000,300))
protos=np.random.normal(size=(500,300))

def numpydists(feats,protos):
  #shapes: are (N,D), (P,D)

  t1 = np.sum(feats * feats, axis=1)[:, np.newaxis] # shape [5000, 1]
  t2 = 2 * feats @ protos.T # shape [5000, 500]
  t3 = np.sum(protos * protos, axis=1)[np.newaxis, :] # shape [1, 500]

  return t1 - t2 + t3

dists = numpydists(feats,protos)
print(dists.shape)

(5000, 500)


In [73]:
feats=np.random.normal(size=(5000,300))
protos=np.random.normal(size=(500,300))

dists_for0 = forloopdists(feats,protos)
dists_for = torch.from_numpy(dists_for0)

dists_numpy0 = numpydists(feats,protos)
dists_numpy = torch.from_numpy(dists_numpy0)

dists_torch = pytorchdists(feats,protos)
dists_torch = dists_torch.to(torch.float64)

print("for and torch close? " + str(torch.allclose(dists_for, dists_torch)))
print("for and numpy close? " + str(torch.allclose(dists_for, dists_numpy)))

for and torch close? True
for and numpy close? True


In [70]:
def run():

  ########
  ##
  ## if you have less than 8 gbyte, then reduce from 250k
  ##
  ###############
  feats=np.random.normal(size=(5000,300)) #5000 instead of 250k for forloopdists
  protos=np.random.normal(size=(500,300))


  since = time.time()
  dists0 = forloopdists(feats,protos)
  time_elapsed=float(time.time()) - float(since)
  print('forloopdists with 5000 samples complete in {:.3f}s'.format( time_elapsed ))
  print(dists0.shape)


  feats=np.random.normal(size=(250000,300))
  protos=np.random.normal(size=(500,300))



  device=torch.device('cpu')
  since = time.time()

  dists1=pytorchdists(feats,protos,device)


  time_elapsed=float(time.time()) - float(since)

  print('pytorchdists with 250000 samples complete in {:.3f}s'.format( time_elapsed ))
  print(dists1.shape)

  #print('df0',np.max(np.abs(dists1-dists0)))


  since = time.time()

  dists2=numpydists(feats,protos)


  time_elapsed=float(time.time()) - float(since)

  print('numpydists with 250000 samples complete in {:.3f}s'.format( time_elapsed ))

  print(dists2.shape)

  print('df',np.max(np.abs(dists1.numpy()-dists2)))


if __name__=='__main__':
  run()


forloopdists with 5000 samples complete in 6.135s
(5000, 500)
pytorchdists with 250000 samples complete in 0.177s
torch.Size([250000, 500])
numpydists with 250000 samples complete in 0.707s
(250000, 500)
df 0.0002327382724160998
